In [14]:
from pathlib import Path
import pandas as pd
import json


## Evaluation of different allocation approaches

In [15]:
directory = Path("evaluation")

In [16]:
# Get list of all considered files
files = []
for filepath in directory.rglob("*"):  # Matches all files and directories
    if filepath.is_file():
        files.append(filepath)
files

[PosixPath('evaluation/paper_process_short_invalids/all_instance_cp.json'),
 PosixPath('evaluation/paper_process_short_invalids/single_instance_cp_warm.json'),
 PosixPath('evaluation/paper_process_short_invalids/single_instance_cp.json'),
 PosixPath('evaluation/paper_process_short_invalids/heuristic.json'),
 PosixPath('evaluation/paper_process_first_invalids/all_instance_cp.json'),
 PosixPath('evaluation/paper_process_first_invalids/single_instance_cp_warm.json'),
 PosixPath('evaluation/paper_process_first_invalids/single_instance_cp.json'),
 PosixPath('evaluation/paper_process_first_invalids/heuristic.json')]

In [17]:
data = {}
for file in files:
    with open(file) as f:
        object = json.load(f)
    data[str(file)] = object["solution"]


In [18]:
df = pd.DataFrame.from_dict(data, orient="index")
df.sort_values(by="objective")

,objective,optimality gap,computing time,solver status,branches,propagations,total interval length
evaluation/paper_process_short_invalids/all_instance_cp.json,40.0,N/A,329.320000,Feasible,3020865.0,N/A,82.0
evaluation/paper_process_short_invalids/single_instance_cp_warm.json,41.0,N/A,199.740000,Optimal,457974.0,N/A,53.0
evaluation/paper_process_short_invalids/single_instance_cp.json,41.0,N/A,306.190000,Optimal,525125.0,N/A,53.0
evaluation/paper_process_short_invalids/heuristic.json,42.0,NaN,7.427491,NaN,NaN,NaN,54.0
evaluation/paper_process_first_invalids/all_instance_cp.json,130.0,N/A,0.070000,Optimal,70635.0,N/A,292.0
evaluation/paper_process_first_invalids/single_instance_cp_warm.json,130.0,N/A,0.050000,Optimal,42809.0,N/A,246.0
evaluation/paper_process_first_invalids/single_instance_cp.json,130.0,N/A,0.070000,Optimal,63128.0,N/A,247.0
evaluation/paper_process_first_invalids/heuristic.json,130.0,NaN,0.057677,NaN,NaN,NaN,247.0
